# Exercise 2
This exercise has the goal to determine ASes that are using the RPKI to perform Route-Origin-Validation (ROV) and filter invalid announcements. 

An invalid announcement is a BGP announcement that has a ROA covering the announced prefix space, but the Origin AS announcing the prefix is not contained in any covering ROA, therefore rendering the announcement invalid. This could be a possible hijack or a misconfiguration.

Methodology: We announce an anchor prefix 147.28.240.0/24 which is always validating to valid. We also announce an experiment prefix 147.28.241.0/24 which is invalid between 04-12 UTC, else valid. Therefore, we are able to observe route changes/missing routes for the experiment prefix once the valitity status changes.

You get all VantagePoint (VP) data as input. A VP is defined as VP ASN + IP. It is structured as follows:

34224 + 94.156.252.18<br>
&emsp; 147.28.240.0/24<br>
&emsp; &emsp; 1586822400: 34224 47065<br>
&emsp; &emsp; 1586829600: 34224 47065<br>
&emsp; &emsp; 1586836800: 34224 47065<br>
&emsp; &emsp; 1586844000: 34224 47065<br>
&emsp; 147.28.241.0/24<br>
&emsp; &emsp; 1586822400: 34224 47065<br>
&emsp; &emsp; 1586829600: 34224 47065<br>
&emsp; &emsp; 1586836800: 34224 47065<br>
&emsp; &emsp; 1586844000: 34224 8455 8283 47065<br>

We observe a route change for the experiment prefix at 1586844000 due to RPKI. 

**Your task: Implement the methodology to find such route changes and identify ROV-enforcing ASes.**

In [1]:
import bgpReader_util.bgp as bgp
import sys
sys.path.insert(1, 'Exercise_2/')
from controlled import *

The imported code snippet renders the data for you and provides you with a nested dict to work on.
The code can be found here: http://localhost:8888/edit/Exercise_2/controlled.py

In [2]:
vp_routes = get_bgp_data_from_file('Exercise_2/bgpdata_exercise_2.txt') #Reads BGP data into nested dict
vp_routes = add_missing_routes(vp_routes) #Adds "missing" if anchor prefix was present at a timepoint but experiment prefix not

p_a = '147.28.240.0/24' #Prefix Anchor
p_e = '147.28.241.0/24' #Prefix Experiment

# vp_routes[vp][prefix][timestamp]
for vp in vp_routes:
    vp_asn = vp[0]
    vp_ip = vp[1]
    
    print()
    print('VantagePoint ASN+IP: ', vp[0] + ' + ' + vp[1])
    for prefix in vp_routes[vp]:
        print(prefix)
        for timestamp in vp_routes[vp][prefix]:
            print(timestamp, vp_routes[vp][prefix][timestamp])



VantagePoint ASN+IP:  18106 + 202.73.40.45
147.28.240.0/24
1586822400 18106 47065
1586829600 18106 47065
1586836800 18106 47065
1586844000 18106 47065
147.28.241.0/24
1586822400 18106 47065
1586829600 18106 47065
1586836800 18106 47065
1586844000 18106 47065

VantagePoint ASN+IP:  54728 + 140.192.8.16
147.28.240.0/24
1586822400 54728 20130 6939 47065
1586829600 54728 20130 6939 47065
1586836800 54728 20130 6939 47065
1586844000 54728 20130 6939 47065
147.28.241.0/24
1586822400 54728 20130 6939 47065
1586829600 54728 20130 6939 47065
1586836800 54728 20130 6939 47065
1586844000 54728 20130 6939 3491 57866 8283 47065

VantagePoint ASN+IP:  34224 + 94.156.252.18
147.28.240.0/24
1586822400 34224 47065
1586829600 34224 47065
1586836800 34224 47065
1586844000 34224 47065
147.28.241.0/24
1586822400 34224 47065
1586829600 34224 47065
1586836800 34224 47065
1586844000 34224 8455 8283 47065

VantagePoint ASN+IP:  49788 + 91.218.184.60
147.28.240.0/24
1586822400 49788 12552 47065
1586829600 4978

Implement your code here:

In [3]:
day = '2020-04-14'
tutorial_results = [] # This array should contain the ASes you identified as ROV-enforcing
for vp in vp_routes:
    vp_asn = vp[0]
    vp_ip = vp[1]
    
    direct_route = str(vp_asn) + ' 47065' #Connectivity requirement

    if p_a not in vp_routes[vp]: continue    #Make sure anchor prefix is present
    
    # If VP has constant, direct, route to p_a..
    if all(vp_routes[vp][p_a][timestamp] == direct_route for timestamp in vp_routes[vp][p_a]):
        if not all(vp_routes[vp][p_e][timestamp] == direct_route for timestamp in vp_routes[vp][p_e]):
            
            # VP has constant, direct route to P_a, but not to P_e
            tutorial_result = (day, vp_asn, vp_ip, p_a, p_e)
            tutorial_results.append(tutorial_result)

                
for i in tutorial_results:
    print(i)


('2020-04-14', '34224', '94.156.252.18', '147.28.240.0/24', '147.28.241.0/24')
('2020-04-14', '31019', '91.228.151.1', '147.28.240.0/24', '147.28.241.0/24')
('2020-04-14', '37100', '105.16.0.247', '147.28.240.0/24', '147.28.241.0/24')
('2020-04-14', '8492', '85.114.0.217', '147.28.240.0/24', '147.28.241.0/24')
('2020-04-14', '6939', '64.71.137.241', '147.28.240.0/24', '147.28.241.0/24')
